In [3]:
from dotenv import load_dotenv
import os
from pathlib import Path
import requests
import pandas as pd
import time

# Load environment variables from .env file
load_dotenv()

# API configuration
api_access_key = os.getenv('API_ACCESS_KEY')
api_secret_key = os.getenv('API_SECRET_KEY')
output_csv_file = Path.cwd().parent / os.getenv('PAINTINGS_CSV_FILE')

In [ ]:
def authenticate(api_access_key, api_secret_key):
    """Authenticate with the WikiArt API and return a session key."""
    login_url = "https://www.wikiart.org/en/Api/2/login"
    params = {"accessCode": api_access_key, "secretCode": api_secret_key}
    response = requests.get(login_url, params=params)
    if response.status_code == 200:
        return response.json()['SessionKey']
    raise Exception(f"Authentication failed: {response.text}")

def fetch_paintings(session_key, pagination_token=''):
    """Fetch a batch of paintings using the session key and handle pagination."""
    url = "https://www.wikiart.org/en/api/2/MostViewedPaintings"
    headers = {"authSessionKey": session_key}
    params = {"paginationToken": pagination_token} if pagination_token else {}
    response = requests.get(url, headers=headers, params=params)
    return response.json()['data'], response.json().get('paginationToken'), response.json().get('hasMore', False) if response.status_code == 200 else ([], None, False)

def fetch_painting_details(painting_id, session_key):
    """Fetch detailed information for a specific painting."""
    url = f"https://www.wikiart.org/en/api/2/Painting?id={painting_id}"
    headers = {"authSessionKey": session_key}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {}

def append_to_csv(paintings, filename):
    """Append retrieved painting details to a CSV file."""
    df = pd.DataFrame(paintings)
    if Path(filename).exists():
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, mode='w', header=True, index=False)

def extract_painting_data(painting):
    """Extract and structure painting data into a dictionary."""
    return {
        'Name of Painting': painting.get('title'),
        'Year Painted': painting.get('completitionYear'),
        'Description': painting.get('description'),
        'Dimensions': f"{painting.get('width')} x {painting.get('height')}",
        'Image URL': painting.get('image'),
        'Full Name of Artist': painting.get('artistName'),
        'Genre of Artist': ', '.join(painting.get('genres', [])),
        'Bio of Artist': painting.get('biography'),
        'Birth Year of Artist': painting.get('birthDay'),
        'Death Year of Artist': painting.get('deathDay'),
        'Artist’s Style': ', '.join(painting.get('styles', []))
    }

# Main execution logic
session_key = authenticate(api_access_key, api_secret_key)
has_more = True
pagination_token = ''
requests_count = 0

while has_more:
    if requests_count >= 400:
        print("Hourly rate limit approached, pausing for an hour.")
        time.sleep(3600)  # Sleep for one hour to reset hourly rate limit
        requests_count = 0  # Reset request count after pausing

    paintings, pagination_token, has_more = fetch_paintings(session_key, pagination_token)
    detailed_paintings = []
    for painting in paintings:
        if requests_count >= 400:
            break  # Check rate limit within the loop to prevent exceeding limits
        details = fetch_painting_details(painting['id'], session_key)
        detailed_paintings.append(extract_painting_data(details))
        requests_count += 1
        time.sleep(0.25)  # Maintain 4 requests per second limit
    append_to_csv(detailed_paintings, output_csv_file)
    print(f"Appended {len(detailed_paintings)} paintings to the dataset.")
    time.sleep(9)  # Manage the 400 requests per hour limit


In [2]:
def authenticate(api_access_key, api_secret_key):
    """Authenticate with the WikiArt API and return a session key."""
    login_url = "https://www.wikiart.org/en/Api/2/login"
    params = {"accessCode": api_access_key, "secretCode": api_secret_key}
    response = requests.get(login_url, params=params)
    if response.status_code == 200:
        return response.json()['SessionKey']
    raise Exception(f"Authentication failed: {response.text}")

def fetch_paintings(session_key):
    """Fetch a single batch of paintings using the session key for testing."""
    url = "https://www.wikiart.org/en/api/2/MostViewedPaintings"
    headers = {"authSessionKey": session_key}
    response = requests.get(url, headers=headers)
    return response.json()['data'][0] if response.status_code == 200 else {}

def fetch_painting_details(painting_id, session_key):
    """Fetch detailed information for a specific painting."""
    url = f"https://www.wikiart.org/en/api/2/Painting?id={painting_id}"
    headers = {"authSessionKey": session_key}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {}

def append_to_csv(painting, filename):
    """Append retrieved painting details to a CSV file."""
    df = pd.DataFrame([painting])
    if Path(filename).exists():
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, mode='w', header=True, index=False)

def extract_painting_data(painting):
    """Extract and structure painting data into a dictionary."""
    return {
        'Name of Painting': painting.get('title'),
        'Year Painted': painting.get('completitionYear'),
        'Description': painting.get('description'),
        'Dimensions': f"{painting.get('width')} x {painting.get('height')}",
        'Image URL': painting.get('image'),
        'Full Name of Artist': painting.get('artistName'),
        'Genre of Artist': ', '.join(painting.get('genres', [])),
        'Bio of Artist': painting.get('biography'),
        'Birth Year of Artist': painting.get('birthDay'),
        'Death Year of Artist': painting.get('deathDay'),
        'Artist’s Style': ', '.join(painting.get('styles', []))
    }

# Main execution logic
session_key = authenticate(api_access_key, api_secret_key)
single_painting = fetch_paintings(session_key)
painting_details = fetch_painting_details(single_painting['id'], session_key)
detailed_painting = extract_painting_data(painting_details)
append_to_csv(detailed_painting, output_csv_file)

print(f"Test complete. Data appended for one painting: {detailed_painting['Name of Painting']}")


Test complete. Data appended for one painting: Mona Lisa


In [12]:
def authenticate():
    """Authenticate with the WikiArt API and return a session key."""
    login_url = "https://www.wikiart.org/en/Api/2/login"
    params = {"accessCode": api_access_key, "secretCode": api_secret_key}
    response = requests.get(login_url, params=params)
    if response.status_code == 200:
        return response.json()['SessionKey']
    raise Exception(f"Authentication failed: {response.text}")

def fetch_paintings(session_key):
    """Fetch a single batch of paintings using the session key for testing."""
    url = "https://www.wikiart.org/en/api/2/MostViewedPaintings"
    headers = {"authSessionKey": session_key}
    response = requests.get(url, headers=headers)
    return response.json()['data'][0] if response.status_code == 200 else {}

def fetch_painting_details(painting_id, session_key):
    """Fetch detailed information for a specific painting."""
    url = f"https://www.wikiart.org/en/api/2/Painting?id={painting_id}"
    headers = {"authSessionKey": session_key}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {}

def extract_painting_data(painting):
    """Extract and structure painting data into a dictionary."""
    return {
        'ID': painting.get('id'),
        'Title': painting.get('title'),
        'Artist Name': painting.get('artistName'),
        'Completion Year': painting.get('completitionYear'),
        'Image URL': painting.get('image'),
        'Width': painting.get('width'),
        'Height': painting.get('height'),
        'Genres': ', '.join(painting.get('genres', [])),
        'Styles': ', '.join(painting.get('styles', [])),
        'Artist ID': painting.get('artistId'),
        'Description': painting.get('description'),
        'Location': painting.get('location')
    }

def append_to_csv(painting_data, filename):
    """Append retrieved painting details to a CSV file."""
    df = pd.DataFrame([painting_data])
    df.to_csv(filename, mode='a', header=not Path(filename).exists(), index=False)

# Main execution logic
session_key = authenticate()
single_painting = fetch_paintings(session_key)
painting_details = fetch_painting_details(single_painting['id'], session_key)
detailed_painting = extract_painting_data(painting_details)
append_to_csv(detailed_painting, output_csv_file)

print(f"Test complete. Data appended for one painting: {detailed_painting['Title']}")

Test complete. Data appended for one painting: Mona Lisa


In [10]:
session_key

'41c7290970f9'

In [20]:
def authenticate():
    """Authenticate with the WikiArt API and return a session key."""
    login_url = "https://www.wikiart.org/en/Api/2/login"
    params = {"accessCode": api_access_key, "secretCode": api_secret_key}
    response = requests.get(login_url, params=params)
    if response.status_code == 200:
        return response.json()['SessionKey']
    raise Exception(f"Authentication failed: {response.text}")

def fetch_paintings(session_key, pagination_token=''):
    """Fetch a batch of paintings using the session key and handle pagination."""
    url = "https://www.wikiart.org/en/api/2/MostViewedPaintings"
    headers = {"authSessionKey": session_key}
    params = {"paginationToken": pagination_token} if pagination_token else {}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        try:
            data = response.json()
            return data['data'], data.get('paginationToken'), data.get('hasMore', False)
        except KeyError:
            print("KeyError: The JSON response does not contain the expected keys. Here's the response:")
            print(response.json())
            return ([], None, False)
    else:
        print(f"Failed to fetch paintings: Status Code {response.status_code}")
        print("Response:", response.text)
        return ([], None, False)

def fetch_painting_details(painting_id, session_key):
    """Fetch detailed information for a specific painting."""
    url = f"https://www.wikiart.org/en/api/2/Painting?id={painting_id}"
    headers = {"authSessionKey": session_key}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {}

def extract_painting_data(painting):
    """Extract and structure painting data into a dictionary."""
    return {
        'ID': painting.get('id'),
        'Title': painting.get('title'),
        'Artist Name': painting.get('artistName'),
        'Completion Year': painting.get('completitionYear'),
        'Image URL': painting.get('image'),
        'Width': painting.get('width'),
        'Height': painting.get('height'),
        'Genres': ', '.join(painting.get('genres', [])),
        'Styles': ', '.join(painting.get('styles', [])),
        'Artist ID': painting.get('artistId'),
        'Description': painting.get('description'),
        'Location': painting.get('location')
    }

def append_to_csv(paintings, filename):
    """Append retrieved painting details to a CSV file."""
    df = pd.DataFrame(paintings)
    df.to_csv(filename, mode='a', header=not Path(filename).exists(), index=False)

# Main execution logic
session_key = authenticate()
has_more = True
pagination_token = ''
requests_count = 0
all_paintings_data = []

while has_more and requests_count < 400:
    paintings, pagination_token, has_more = fetch_paintings(session_key, pagination_token)
    for painting in paintings:
        painting_details = fetch_painting_details(painting['id'], session_key)
        if painting_details:
            detailed_painting = extract_painting_data(painting_details)
            all_paintings_data.append(detailed_painting)
            requests_count += 1
            if requests_count >= 400:
                break
            time.sleep(0.25)  # Respect the rate limit of 4 requests per second

append_to_csv(all_paintings_data, output_csv_file)
print(f"Appended {len(all_paintings_data)} paintings to the dataset.")

if has_more:
    print("Not all paintings processed due to rate limit. Run again after sufficient time has passed.")

Failed to fetch paintings: Status Code 500
Response: {"Exception":{"Message":"Padding is invalid and cannot be removed.","Stacktrace":null,"InnerException":null},"Status":500,"RequestTime":"\/Date(1715126871891)\/"}
Appended 60 paintings to the dataset.


In [22]:
def authenticate():
    """Authenticate with the WikiArt API and return a session key."""
    login_url = "https://www.wikiart.org/en/Api/2/login"
    params = {"accessCode": api_access_key, "secretCode": api_secret_key}
    response = requests.get(login_url, params=params)
    if response.status_code == 200:
        return response.json()['SessionKey']
    raise Exception(f"Authentication failed: {response.text}")

def robust_fetch(url, headers, params, retries=3, backoff_factor=0.5):
    """Fetch data with retries and exponential backoff."""
    for attempt in range(retries):
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        elif response.status_code >= 500:  # Server error
            time.sleep(backoff_factor * (2 ** attempt))
        else:
            break  # Break if the error is client-side
    return None  # Return None if all retries fail

# Update the fetch_paintings function to use robust_fetch
def fetch_paintings(session_key, pagination_token=''):
    url = "https://www.wikiart.org/en/api/2/MostViewedPaintings"
    headers = {"authSessionKey": session_key}
    params = {"paginationToken": pagination_token} if pagination_token else {}
    response = robust_fetch(url, headers, params)
    if response:
        return response.get('data', []), response.get('paginationToken'), response.get('hasMore', False)
    return ([], None, False)

def fetch_painting_details(painting_id, session_key):
    """Fetch detailed information for a specific painting."""
    url = f"https://www.wikiart.org/en/api/2/Painting?id={painting_id}"
    headers = {"authSessionKey": session_key}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {}

def extract_painting_data(painting):
    """Extract and structure painting data into a dictionary."""
    return {
        'ID': painting.get('id'),
        'Title': painting.get('title'),
        'Artist Name': painting.get('artistName'),
        'Completion Year': painting.get('completitionYear'),
        'Image URL': painting.get('image'),
        'Width': painting.get('width'),
        'Height': painting.get('height'),
        'Genres': ', '.join(painting.get('genres', [])),
        'Styles': ', '.join(painting.get('styles', [])),
        'Artist ID': painting.get('artistId'),
        'Description': painting.get('description'),
        'Location': painting.get('location')
    }

def append_to_csv(paintings, filename):
    """Append retrieved painting details to a CSV file."""
    df = pd.DataFrame(paintings)
    df.to_csv(filename, mode='a', header=not Path(filename).exists(), index=False)

# Main execution logic
# session_key = authenticate()
has_more = True
pagination_token = ''
requests_count = 0
all_paintings_data = []

while has_more and requests_count < 400:
    paintings, pagination_token, has_more = fetch_paintings(session_key, pagination_token)
    print(f"Fetched {len(paintings)} paintings, hasMore: {has_more}, paginationToken: {pagination_token}")
    for painting in paintings:
        painting_details = fetch_painting_details(painting['id'], session_key)
        if painting_details:
            detailed_painting = extract_painting_data(painting_details)
            all_paintings_data.append(detailed_painting)
            requests_count += 1
            if requests_count >= 400:
                break
            time.sleep(0.25)  # Respect the rate limit of 4 requests per second

append_to_csv(all_paintings_data, output_csv_file)
print(f"Appended {len(all_paintings_data)} paintings to the dataset.")

if has_more:
    print("Not all paintings processed due to rate limit. Run again after sufficient time has passed.")

Fetched 60 paintings, hasMore: True, paginationToken: XmiPhej%2f%2bV5BvhAp2w%2frnN3%2bxsFO4FcgvvvZFwFz5oI%2bqkvpjnXlzIVoR1XOP%2bY%2bifsOuu7WjmaxySEuiBTdx0D2BiSO9dd%2bqJpDcwWZXOk%3d
Fetched 0 paintings, hasMore: False, paginationToken: None
Appended 60 paintings to the dataset.


In [23]:
# Specify the path to your CSV file
csv_file_path = 'paintings_dataset.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Check the first few rows of the DataFrame to understand its structure
print(df.head())

# Extract the 'Artist Name' column and find unique names
unique_artists = df['Artist Name'].unique()

# Print the list of unique artist names
print("Unique Artists in the Dataset:")
print(unique_artists)
print(f"Total unique artists: {len(unique_artists)}")

                         ID                 Title                Artist Name  \
0  57727444edc2cb3880cb7bf6             Mona Lisa          da Vinci Leonardo   
1  5772716cedc2cb3880c1907f      The Starry Night          van Gogh Vincent    
2  57727593edc2cb3880ceb255      In Bed, The Kiss  Toulouse-Lautrec Henri de   
3  57726e7bedc2cb3880b7466a    The Birth of Venus         Botticelli Sandro    
4  577271f9edc2cb3880c37dcd  The School of Athens                    Raphael   

   Completion Year                                          Image URL  Width  \
0           1519.0  https://uploads5.wikiart.org/00475/images/leon...    408   
1           1889.0  https://uploads3.wikiart.org/00475/images/vinc...    750   
2           1892.0  https://uploads8.wikiart.org/images/henri-de-t...    750   
3           1485.0  https://uploads6.wikiart.org/images/sandro-bot...    750   
4           1511.0  https://uploads6.wikiart.org/00475/images/raph...    750   

   Height                 Genres      

In [26]:
csv_file_path = 'paintings_dataset.csv'
output_csv_file = 'paintings_dataset.csv'

def authenticate():
    """Authenticate with the WikiArt API and return a session key."""
    login_url = "https://www.wikiart.org/en/Api/2/login"
    params = {"accessCode": api_access_key, "secretCode": api_secret_key}
    response = requests.get(login_url, params=params)
    if response.status_code == 200:
        return response.json()['SessionKey']
    raise Exception(f"Authentication failed: {response.text}")

def fetch_paintings_by_artist(session_key, artist_id, pagination_token=''):
    """Fetch paintings by artist ID using the session key and handle pagination."""
    url = f"https://www.wikiart.org/en/api/2/PaintingsByArtist?id={artist_id}"
    headers = {"authSessionKey": session_key}
    params = {"paginationToken": pagination_token} if pagination_token else {}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['data'], data.get('paginationToken'), data.get('hasMore', False)
    else:
        print(f"Failed to fetch paintings for artist {artist_id}: {response.text}")
        return ([], None, False)

def append_to_csv(paintings, filename):
    """Append retrieved painting details to a CSV file."""
    df = pd.DataFrame(paintings)
    df.to_csv(filename, mode='a', header=not Path(filename).exists(), index=False)

# Load the existing CSV and extract unique artist IDs
df = pd.read_csv(csv_file_path)
unique_artist_ids = df['Artist ID'].unique()

# Authenticate to get a session key
session_key = authenticate()
requests_count = 0

# Loop through each unique artist ID and fetch their paintings
for artist_id in unique_artist_ids:
    has_more = True
    pagination_token = ''
    while has_more and requests_count < 400:
        paintings, pagination_token, has_more = fetch_paintings_by_artist(session_key, artist_id, pagination_token)
        if paintings:
            append_to_csv(paintings, output_csv_file)
            requests_count += len(paintings)
            print(f"Appended {len(paintings)} paintings for artist ID {artist_id}")
        time.sleep(0.25)  # Sleep to respect the rate limit of 4 requests per second
        if requests_count >= 400:
            print("Hourly limit reached. Sleeping for one hour.")
            time.sleep(3600)  # Sleep for one hour to reset the limit
            requests_count = 0

print("Completed appending all paintings for listed artists.")

Appended 60 paintings for artist ID 57726d85edc2cb3880b48ccd
Failed to fetch paintings for artist 57726d85edc2cb3880b48ccd: {"Exception":{"Message":"Padding is invalid and cannot be removed.","Stacktrace":null,"InnerException":null},"Status":500,"RequestTime":"\/Date(1715130086816)\/"}
Appended 60 paintings for artist ID 57726d82edc2cb3880b486a0
Failed to fetch paintings for artist 57726d82edc2cb3880b486a0: {"Exception":{"Message":"Padding is invalid and cannot be removed.","Stacktrace":null,"InnerException":null},"Status":500,"RequestTime":"\/Date(1715130088672)\/"}
Appended 60 paintings for artist ID 57726d86edc2cb3880b48f5d
Failed to fetch paintings for artist 57726d86edc2cb3880b48f5d: {"Exception":{"Message":"Padding is invalid and cannot be removed.","Stacktrace":null,"InnerException":null},"Status":500,"RequestTime":"\/Date(1715130090338)\/"}
Appended 60 paintings for artist ID 57726d7dedc2cb3880b47c88
Failed to fetch paintings for artist 57726d7dedc2cb3880b47c88: {"Exception":{"

KeyboardInterrupt: 

In [2]:
import requests
import json

def fetch_artists():
    base_url = "https://www.wikiart.org"
    url = f"{base_url}/en/App/Artist/AlphabetJson"
    params = {'v': 'new', 'inPublicDomain': 'true'}
    try:
        response = requests.get(url, params=params, timeout=10)  # Including a timeout as good practice
        response.raise_for_status()  # This will raise an exception for HTTP errors
        artists = response.json()
        print(f"Found {len(artists)} artists.")
        return artists
    except requests.exceptions.HTTPError as e:
        print("HTTP Error:", e)
        return []
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return []
    except json.decoder.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        return []

# Test fetching artists
artists = fetch_artists()

Found 5665 artists.


In [3]:
artists[0]

{'contentId': 9223372032559910281,
 'artistName': '20240414 180739',
 'url': '20240414-180739',
 'lastNameFirst': None,
 'birthDay': '/Date(-62135510402840)/',
 'deathDay': '/Date(253402300799999)/',
 'birthDayAsString': 'April 9, 1840 BC',
 'deathDayAsString': None,
 'image': 'https://uploads4.wikiart.org/00493/images//20240414-180739.jpg!Portrait.jpg',
 'wikipediaUrl': None,
 'dictonaries': []}

In [4]:
def fetch_paintings_by_artist(artist_id):
    base_url = "https://www.wikiart.org"
    url = f"{base_url}/en/App/PaintingsByArtist/{artist_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP request errors
        paintings = response.json()
        print(f"Found {len(paintings)} paintings for artist ID {artist_id}.")
        return paintings
    except requests.exceptions.HTTPError as e:
        print("HTTP Error:", e)
        return []
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return []
    except json.decoder.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        return []

# Example use of the function:
artist_id = artists[0]['contentId']  # Assuming 'contentId' is the correct ID to use
paintings = fetch_paintings_by_artist(artist_id)

HTTP Error: 400 Client Error: Bad Request for url: https://www.wikiart.org/en/App/Home/Error/57736375edc2c90e54dcf1d8


In [5]:
def fetch_paintings_by_artist(artist):
    """Fetch paintings by artist using their URL and save the info as JSON.

    :param artist: dict, containing at least 'url' and 'artistName'.
    """
    base_url = "https://www.wikiart.org"

    url = f"{base_url}/Painting/PaintingsByArtist"
    params = {'artistUrl': artist['url'], 'json': 2}

    print(f"Fetching paintings for {artist['artistName']}")
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        paintings = response.json()

        print(f"Retrieved and saved {len(paintings)} paintings.")
        return paintings
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch paintings: {e}")
        return []

paintings = fetch_paintings_by_artist(artists[0])

Fetching paintings for 20240414 180739
Failed to fetch paintings: 404 Client Error: Not Found for url: https://www.wikiart.org/Painting/PaintingsByArtist?artistUrl=20240414-180739&json=2


In [6]:
def fetch_paintings_by_artist(artist):
    """Fetch paintings by artist using their URL."""
    base_url = "https://www.wikiart.org"

    url = f"{base_url}/Painting/PaintingsByArtist"
    params = {'artistUrl': artist['url'], 'json': 2}

    print(f"Fetching paintings for {artist['artistName']}")
    try:
        response = requests.get(url, params=params, timeout=10)
        print("Request URL:", response.url)  # To see the constructed URL
        response.raise_for_status()
        paintings = response.json()

        print(f"Retrieved {len(paintings)} paintings.")
        return paintings
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        return []
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch paintings: {e}")
        return []

# Test the function
paintings = fetch_paintings_by_artist({'url': 'claude-monet', 'artistName': 'Claude Monet'})

Fetching paintings for Claude Monet
Request URL: https://www.wikiart.org/Painting/PaintingsByArtist?artistUrl=claude-monet&json=2
HTTP Error: 404 Client Error: Not Found for url: https://www.wikiart.org/Painting/PaintingsByArtist?artistUrl=claude-monet&json=2
